In [12]:
## standard and modeling imports

import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import copy
#from Modules import *
#sns.set()
%matplotlib inline
import imblearn
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score,  KFold
from sklearn.ensemble import RandomForestClassifier
#import xgboost as xgb
#from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [5]:
#df = pd.read_excel('Full_Datav16_zip_extract-Logistic.xls', header=None)
df = pd.read_excel('Full_Datav16_zip_extract-Logistic.xls', header=0, skiprows=0,)

df.head()

,Above Median TAM,Total Citizen Educated in US,Citizen Less than High School Education,Citizen High School Graduate,Citizen Some College Education,Citizen College Degree,Citizen Graduate or Professional Degree,Total Citizen Income,IPR < 1.25 Citizen No Income,"IPR < 1.25 Citizen Income $1-$9,999",...,Native American Race,Asian Race,Pacific Islander Race,Other Race Alone,Hispanic or Latino,Median Age,Male Median Age,Female Median Age,Total Households,Total Families
0,0,13358,1167,3779,4397,2397,1618,14915,923,1964,...,357,0,141,229,16209,45.5,43.0,47.7,7215,3098
1,0,13787,633,1684,2229,3405,5836,27181,3470,9966,...,2971,12,729,1217,26766,24.0,23.6,24.8,9910,4050
2,0,136,26,13,68,14,15,11131,1384,8858,...,1195,1,180,372,9843,20.2,20.2,20.1,5,10
3,0,3498,123,1270,1023,547,535,4250,477,450,...,27,0,21,71,4990,41.0,40.0,41.7,1904,1284
4,0,10062,644,2407,2518,2423,2070,12325,1021,1666,...,305,8,115,260,14264,41.1,40.0,42.0,5595,3691


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32936 entries, 0 to 32935
Data columns (total 31 columns):
Above Median TAM                                32936 non-null int64
Total Citizen Educated in US                    32936 non-null int64
Citizen Less than High School  Education        32936 non-null int64
Citizen High School  Graduate                   32936 non-null int64
Citizen Some College  Education                 32936 non-null int64
Citizen College Degree                          32936 non-null int64
Citizen Graduate or Professional Degree         32936 non-null int64
Total Citizen Income                            32936 non-null int64
IPR < 1.25 Citizen No Income                    32936 non-null int64
IPR < 1.25 Citizen Income $1-$9,999             32936 non-null int64
IPR < 1.25 Citizen Income $10,000-$14,999       32936 non-null int64
IPR 1.25-3.49 Citizen Income $15,000-$24,999    32936 non-null int64
IPR 1.25-3.49 Citizen Income $25,000-$34,999    32936 non-null 

In [7]:
# building features and re-naming target data Y

df.rename(index=str, columns={"Above Median TAM": "Y"}, inplace=True)
#df.drop('NAME', axis=1, inplace=True )
#df = df.astype('float32')

df.head()

,Y,Total Citizen Educated in US,Citizen Less than High School Education,Citizen High School Graduate,Citizen Some College Education,Citizen College Degree,Citizen Graduate or Professional Degree,Total Citizen Income,IPR < 1.25 Citizen No Income,"IPR < 1.25 Citizen Income $1-$9,999",...,Native American Race,Asian Race,Pacific Islander Race,Other Race Alone,Hispanic or Latino,Median Age,Male Median Age,Female Median Age,Total Households,Total Families
0,0,13358,1167,3779,4397,2397,1618,14915,923,1964,...,357,0,141,229,16209,45.5,43.0,47.7,7215,3098
1,0,13787,633,1684,2229,3405,5836,27181,3470,9966,...,2971,12,729,1217,26766,24.0,23.6,24.8,9910,4050
2,0,136,26,13,68,14,15,11131,1384,8858,...,1195,1,180,372,9843,20.2,20.2,20.1,5,10
3,0,3498,123,1270,1023,547,535,4250,477,450,...,27,0,21,71,4990,41.0,40.0,41.7,1904,1284
4,0,10062,644,2407,2518,2423,2070,12325,1021,1666,...,305,8,115,260,14264,41.1,40.0,42.0,5595,3691


In [9]:
X = df.drop(columns = 'Y')
y = df['Y']

df.describe()

,Y,Total Citizen Educated in US,Citizen Less than High School Education,Citizen High School Graduate,Citizen Some College Education,Citizen College Degree,Citizen Graduate or Professional Degree,Total Citizen Income,IPR < 1.25 Citizen No Income,"IPR < 1.25 Citizen Income $1-$9,999",...,Native American Race,Asian Race,Pacific Islander Race,Other Race Alone,Hispanic or Latino,Median Age,Male Median Age,Female Median Age,Total Households,Total Families
count,32936.0,32936.000000,32936.000000,32936.000000,32936.000000,32936.000000,32936.000000,32936.000000,32936.000000,32936.000000,...,32936.000000,32936.000000,32936.000000,32936.000000,32936.000000,32936.000000,32936.000000,32936.000000,32936.000000,32936.000000
mean,0.0,6551.727502,831.603716,1791.580034,1905.948658,1252.807839,769.787254,7876.207524,1077.505435,1267.017397,...,444.878917,16.382864,579.313305,272.959467,7823.575146,41.029934,40.172917,41.815782,3535.097917,2248.882894
std,0.0,9585.041286,1746.562909,2547.765106,2830.556224,2239.751193,1546.843108,11580.788052,1863.973525,1920.903748,...,1823.460191,153.952027,2142.990601,602.989144,10930.976279,7.441885,7.596357,7.558833,5049.549402,3532.907282
min,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,487.000000,48.000000,174.000000,142.000000,51.000000,21.000000,584.000000,59.000000,97.000000,...,1.000000,0.000000,3.000000,9.000000,681.000000,37.300000,36.100000,38.200000,283.000000,154.000000
50%,0.0,1919.000000,209.000000,657.000000,568.000000,226.000000,114.000000,2280.000000,264.000000,378.000000,...,10.500000,0.000000,21.500000,41.000000,2552.500000,41.200000,40.300000,42.200000,1069.000000,626.000000
75%,0.0,8916.000000,846.000000,2390.000000,2510.250000,1354.000000,746.000000,10651.000000,1314.000000,1689.000000,...,142.000000,5.000000,213.000000,256.000000,10969.250000,45.000000,44.300000,45.800000,4861.500000,2920.250000
max,0.0,73082.000000,36261.000000,28207.000000,23713.000000,25738.000000,27438.000000,88221.000000,22692.000000,22931.000000,...,60867.000000,14484.000000,52759.000000,20765.000000,86519.000000,86.700000,85.400000,87.400000,44432.000000,38842.000000


In [10]:
##splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 2019, test_size = .25)

In [11]:
#create  oversampled data to train on
oversampler = SMOTE(random_state = 2019)
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

ValueError: The target 'y' needs to have more than 1 class. Got 1 class instead

In [13]:
from imblearn.over_sampling import SMOTE

#create my oversampled data to train on
oversampler = SMOTE(random_state = 2019)
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train, y_train)

ValueError: The target 'y' needs to have more than 1 class. Got 1 class instead

In [ ]:
#Put the oversampled data back into a dataframe
X_train_oversampled = pd.DataFrame(X_train_oversampled, columns = X_train.columns)
y_train_oversampled = pd.Series(y_train_oversampled)

In [ ]:
# build my classifier
from sklearn.linear_model import LinearRegression, LinearRegressionCV
clf = LogisticRegression()

In [ ]:
# build my classifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
clf = LogisticRegression()

In [ ]:
#Fit to the oversampled data; this will train the classifier on the oversampled data
clf.fit(X_train_oversampled, y_train_oversampled)

In [ ]:
#Use 5-fold cross validation to see how well the classfier you built is doing on test data.

kfold = KFold(n_splits=5, random_state=2019)
results = cross_val_score(clf, X_test, y_test, cv=kfold, scoring = 'f1')

print(f"5-fold cross-validation results: {np.mean(results)}")

In [ ]:
#tweak my classifer

clf2 = LogisticRegression(C = .5)

clf2.fit(X_train_oversampled, y_train_oversampled)

results = cross_val_score(clf2, X_test, y_test, cv=kfold, scoring = 'f1')

print(f"5-fold cross-validation results: {np.mean(results)}")

In [ ]:
clf_cv = LogisticRegressionCV(n_jobs = -1)

clf_cv.fit(X_train_oversampled, y_train_oversampled)

results = cross_val_score(clf, X_test, y_test, cv=kfold, scoring = 'f1')

print(f"5-fold cross-validation results: {np.mean(results)}")